In [245]:
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
import subprocess
import shutil
from selenium.webdriver.common.by import By
import pandas as pd
import time

from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC

# 셀레니움 신버전 크롬 드라이버 패키지

from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

from selenium.webdriver.common.keys import Keys
# 진행률
from tqdm.notebook import tqdm
import re
import os

In [293]:
class Crawling_review() :
    def __init__(self , dataframe) :
        self.id_df = dataframe
        columns = ['리뷰' , '리뷰이미지url' , '작성자프로필' , 'My플레이스' , '작성자이름' , '리뷰작성수'
                  , '리뷰작성일' , '식당방문수' , '리뷰인증방식' , '식당id' , '사업장명']
        self.rest_review_df = pd.DataFrame(columns=columns)
        
        
    def find_img_url(self , rest_img) :
        rest_img_urls = []
        for stylejs_parent in rest_img :
            stylejs = stylejs_parent.find_element(by = By.CSS_SELECTOR , value = 'a > .K0PDV._img.fKa0W')
            #print('stylejs = ' , stylejs)
            url_test_data = stylejs.get_attribute('style')
            #print('data = ' , url_test_data)
            url_find_num = url_test_data.find('url("')

            if 'jpg' in url_test_data :
                url_find_num_end = url_test_data.find('jpg')
            elif 'jpe' in url_test_data :
                url_find_num_end = url_test_data.find('peg')
            else :
                url_find_num_end = url_test_data.find('.")')

            rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]

            if rest_img_url != '' :
                rest_img_urls.append(rest_img_url)

        return rest_img_urls
    
    def naver_rest_review(self , rest_name , rest_id) :    

        name = rest_name
        id_num = rest_id

        # Chrome headless 설정
        options = Options()
        # options.add_argument('--headless')

        # 드라이버 생성
        driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

        # name = '고기다 성산흑돼지'
        # 리뷰 페이지 열기
        print(f'현재 크롤링이 진행중인 검색어 : {name}')

        url = f"https://map.naver.com/v5/search/{name}/place/{id_num}"

        driver.get(url)
        wait = WebDriverWait(driver, 5)
        iframe = wait.until(EC.presence_of_element_located((By.ID, "entryIframe")))
        driver.switch_to.frame(iframe)

        # 리뷰 클릭
        # veBoZ class tag에 해당하는 요소가 존재하는지 확인
        try :
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "veBoZ")))
            category = driver.find_elements(by = By.CLASS_NAME , value = 'veBoZ')
            tlst = [c.text for c in category]
            if '리뷰' not in tlst :
                
                df = pd.DataFrame(
                    {'리뷰' : None 
                     , '리뷰이미지url' : None  
                     , '작성자프로필' : None 
                     , 'My플레이스' : None 
                     , '작성자이름' : None  
                     , '리뷰작성수' : None 
                     , '리뷰작성일' : None 
                     , '식당방문수' : None 
                     , '리뷰인증방식' : None 
                    } , index=[0])
                df['식당id'] = rest_id
                df['사업장명'] = rest_name
                
                return df
            else :
                for num , tag in enumerate(category) :
                    # print(tag.text)
                    if tag.text == '리뷰' :
                        driver.find_elements(by = By.CLASS_NAME
                                            , value = 'veBoZ')[num].click()
        except :
            pass

        # 최신순 클릭
        # mSdTM class tag가 선택 가능한지 확인

        try :
            time.sleep(1.5)
            wait.until(EC.presence_of_element_located((By.CLASS_NAME, "mSdTM")))
            # wait.until(EC.element_to_be_clickable((By.CLASS_NAME , "mSdTM")))
            driver.find_elements(by = By.CLASS_NAME , value = "mSdTM")[1].click()
        except :
            pass


        # 리뷰 전체 펼치기
        try :
            wait.until(EC.presence_of_all_elements_located((By.CLASS_NAME , "fvwqf")))
            click = driver.find_element(by = By.CLASS_NAME , value = 'fvwqf')

            while click is not None:
                try :
                    click.click()  
                except :
                    break
        except :
            print('펼칠 리뷰가 없다.')
            pass

        # driver 객체를 사용하여 body 태그를 찾고, Keys.HOME을 사용하여 스크롤을 맨 위로 이동합니다.
        # fvwqf class tag가 보이지 않으면 진행합니다.
        try :
            wait.until(EC.invisibility_of_element_located((By.CLASS_NAME , "fvwqf")))
        except :
            pass
        body = driver.find_element(by = By.TAG_NAME , value = 'body')
        body.send_keys(Keys.HOME)
        time.sleep(0.4)
        review_click = driver.find_elements(by = By.CLASS_NAME , value = 'rvCSr')
        if len(review_click) != 0 :
            for address in tqdm(review_click) :
                address.click()
                time.sleep(0.2)
        else :
            pass

        # 리뷰 가져오기
        # 현재는 리뷰만 가져오지만 가져올 데이터를 어떤 것을 정하느냐에 따라 더 많은 정보를 가져올 수도 있다.

        # 리뷰의 여러 정보들을 담을 변수 선언
        
        review_text = []
        review_image = []
        review_profile = []
        review_myplace = []
        review_nickname = []
        review_information = []
        review_date = []
        review_visit_count = []
        review_check = []

        review_all = driver.find_elements(by=By.CLASS_NAME , value = 'YeINN')
        
        # 리뷰가 1개 보다 많을 때만 수집
        # 데어프레임 생성시 스칼라로 생성되어 귀찮기 떄문
        
        if len(review_all) > 1 :
            for review_once in tqdm(review_all , '리뷰수집중') :
                
                # 리뷰 텍스트 
                        
                rest_txt_case01 = review_once.find_elements(by=By.CLASS_NAME , value = 'ZZ4OK.IwhtZ > a > .zPfVt')
                rest_txt_case02 = review_once.find_elements(by=By.CLASS_NAME , value = 'ZZ4OK > a > .zPfVt')
                               
                
                if len(rest_txt_case01) != 0 :
                    review_text.append(rest_txt_case01[0].text)
                elif len(rest_txt_case02) != 0 :
                    review_text.append(rest_txt_case02[0].text)
                else :
                    review_text.append('')
                    pass
                
                # 작성자 첨부 이미지
                
                rest_img_case01 = review_once.find_elements(by=By.CLASS_NAME , 
                              value = 'Mmpzx > .ngGKH > .flicking-viewport > .flicking-camera > .ODalI')
                rest_img_case02 = review_once.find_elements(by=By.CLASS_NAME , 
                              value = 'Mmpzx.jQTnr > .ngGKH > .flicking-viewport > .flicking-camera > .ODalI')
                
                
                if len(rest_img_case01) != 0 :
                    RIU = self.find_img_url(rest_img_case01)
                    review_image.append(RIU)
                elif len(rest_img_case02) != 0 :
                    RIU = self.find_img_url(rest_img_case02)
                    review_image.append(RIU)
                else :
                    review_image.append('') 
                
                
                # 작성자 프로필 사진
                try :
                    profile = review_once.find_element(by=By.CLASS_NAME , value ='DqSJm > .place_thumb > img')
                    review_profile.append(profile.get_attribute('src'))
                except :
                    review_profile.append('https://g-place.pstatic.net/assets/shared/images/icon_default_profile.png')

                # 작성자 MY플레이스
                try :
                    my_place = review_once.find_element(by=By.CLASS_NAME , value ='Hazns')
                    review_myplace.append(my_place.get_attribute('href'))
                except :
                    review_myplace.append('')

                # 작성자 닉네임
                try :
                    nickname = review_once.find_element(by=By.CLASS_NAME , value ='Hazns > .sBWyy')
                    review_nickname.append(nickname.text)
                except :
                    review_nickname.append('')

                # 작성자 리뷰 사진 팔로워 수
                reviewer_info = review_once.find_elements(by=By.CLASS_NAME , value ='Hazns > .Qde7Q > .P1zUJ')
                reviewer_info_text_lst = []

                if len(reviewer_info) != 0 :
                    for reviewer_info_text in reviewer_info :
                            #print(reviewer_info_text.text)
                            reviewer_info_text_lst.append(reviewer_info_text.text)
                else : 
                    pass
                
                review_information.append(reviewer_info_text_lst)
                
                
                
                # 오래된 리뷰에는 별점이 달려있어 구분해주어야한다.
                sb8UA_tag = review_once.find_elements(by=By.CLASS_NAME , value='sb8UA > span')
                
                if len(sb8UA_tag) == 3 :

                    # 작성일자
                    date = review_once.find_elements(by=By.CLASS_NAME , value ='sb8UA > .P1zUJ > .place_blind')
                    review_date.append(date[1].text)

                    # 방문 횟수
                    visit_count = review_once.find_elements(by=By.CLASS_NAME , value ='sb8UA > .P1zUJ')
                    review_visit_count.append(visit_count[1].text)

                    # 리뷰 확인 방식
                    check = review_once.find_elements(by=By.CLASS_NAME , value = 'sb8UA > .P1zUJ')
                    review_check.append(check[2].text)
                
                else :
                    try :
                        # 작성일자
                        date = review_once.find_elements(by=By.CLASS_NAME , value ='sb8UA > .P1zUJ > .place_blind')
                        review_date.append(date[2].text)

                        # 방문 횟수
                        visit_count = review_once.find_elements(by=By.CLASS_NAME , value ='sb8UA > .P1zUJ')
                        review_visit_count.append(visit_count[2].text)

                        # 리뷰 확인 방식
                        check = review_once.find_elements(by=By.CLASS_NAME , value = 'sb8UA > span')
                        review_check.append(check[3].text)
                    except :
                        review_date.append('')
                        review_visit_count.append('')
                        review_check.append('')
                
            
            df = pd.DataFrame(
                {'리뷰' : review_text 
                 , '리뷰이미지url' : review_image 
                 , '작성자프로필' : review_profile
                 , 'My플레이스' : review_myplace
                 , '작성자이름' : review_nickname 
                 , '리뷰작성수' : review_information
                 , '리뷰작성일' : review_date
                 , '식당방문수' : review_visit_count
                 , '리뷰인증방식' : review_check
                })
            df['식당id'] = rest_id
            df['사업장명'] = rest_name
            
        else :
            df = pd.DataFrame(
                {'리뷰' : None 
                 , '리뷰이미지url' : None  
                 , '작성자프로필' : None 
                 , 'My플레이스' : None 
                 , '작성자이름' : None  
                 , '리뷰작성수' : None 
                 , '리뷰작성일' : None 
                 , '식당방문수' : None 
                 , '리뷰인증방식' : None 
                } , index=[0])
            df['식당id'] = rest_id
            df['사업장명'] = rest_name
        
        driver.quit()
        
        self.rest_review_df = df
        return df
        
        
    def crawling_review_loop(self) :
        count = 1
        main_frm = self.id_df
        concat_frm = self.rest_review_df
        rest_name_lst = main_frm.사업장명.tolist()
        rest_id_lst = main_frm.식당id.tolist()

        region_01 = self.id_df.시도.iloc[0]
        region_02 = self.id_df.시군구.iloc[0]
        region_03 = self.id_df.읍면동.iloc[0]

        if not os.path.exists(f'{region_01}') :
            os.makedirs(f'{region_01}') 
        else :
            pass
        if not os.path.exists(f'{region_01}/{region_02}') :
            os.makedirs(f'{region_01}/{region_02}') 
        else :
            pass
        if not os.path.exists(f'{region_01}/{region_02}/{region_03}') :
            os.makedirs(f'{region_01}/{region_02}/{region_03}') 
        else :
            pass
        if not os.path.exists(f'{region_01}/{region_02}/식당정보') :
            os.makedirs(f'{region_01}/{region_02}/식당정보') 
        else :
            pass
        
        print('시작')
        print('-'*60)
        for rest_name , rest_id in zip(rest_name_lst , rest_id_lst) :
            print(count)
            print('현재 : ' , rest_name)

            rest_df = self.naver_rest_review(rest_name , rest_id)
            rest_df.to_excel(f'{region_01}/{region_02}/{region_03}/{rest_name}_review.xlsx' 
                             , index=False)

            concat_frm = pd.concat([concat_frm , rest_df] , axis = 0)

            count+=1
            print()
        print('-'*60)
        print('종료')
        
        result_frm = concat_frm.reset_index(drop=True)
        result_frm.to_excel(f'{region_01}/{region_02}/식당정보/{region_01}_{region_02}_{region_03}_{rest_name}_all_review.xlsx' 
                             , index=False)
        return result_frm

In [286]:
data = pd.read_csv('제주특별자치도/제주시/식당정보/제주특별자치도_제주시_연동_식당id.csv' , encoding='euc-kr')

In [289]:
data

,사업장명,식당id,업종구분소분류,상세영업상태명,소재지전체주소,도로명전체주소,도로명우편번호,시도,시도_시군구,시군구,시도_시군구_읍면동,읍면동,시도코드,시군구코드,읍면동코드
0,봉플라봉뱅,1713847517,기타,영업,제주특별자치도 제주시 연동 312-57번지 정인하우스,"제주특별자치도 제주시 문송1길 6-1, 정인하우스 1층 (연동)",63122.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
1,스톤아일랜드탭하우스,1244422132,기타,영업,제주특별자치도 제주시 연동 280-37,"제주특별자치도 제주시 삼무로7길 16, 1층 (연동)",63118.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
2,연화양꼬치,1482063535,중국식,영업,제주특별자치도 제주시 연동 276-3,"제주특별자치도 제주시 연동8길 28, 1층 105,106호 (연동)",63136.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
3,24시누름돌김치찌개,1028483996,기타,영업,제주특별자치도 제주시 연동 304-3,"제주특별자치도 제주시 노연로 146, 1층 (연동)",63133.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
4,영화루,1109507832,중국식,영업,제주특별자치도 제주시 연동 304-37,"제주특별자치도 제주시 사장길 15-1, 1층 (연동)",63133.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
963,크라우드피씨방제주삼무점,1318887932,기타 휴게음식점,영업,제주특별자치도 제주시 연동 260-15,"제주특별자치도 제주시 신광로 16, 1층 101호 (연동)",63125.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
964,만가복,1880272443,일반조리판매,영업,제주특별자치도 제주시 연동 252-13,"제주특별자치도 제주시 연동1길 35, 1층 (연동)",63131.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
965,토라네코제주본점,1778945379,일반조리판매,영업,제주특별자치도 제주시 연동 300-3 남해오네뜨,"제주특별자치도 제주시 삼무로11길 9, 1층 103호 (연동, 남해오네뜨)",63118.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700
966,귀아랑2길베이커리카페,1887309829,커피숍,영업,제주특별자치도 제주시 연동 311-25,"제주특별자치도 제주시 귀아랑2길 8, 1층 (연동)",63119.0,제주특별자치도,제주특별자치도 제주시,제주시,제주특별자치도 제주시 연동,연동,5000000000,5011000000,5011013700


In [269]:
test = Crawling_review(data[:5])

In [270]:
test_01 = test.crawling_review_loop()

시작
------------------------------------------------------------
1
현재 :  봉플라봉뱅
현재 크롤링이 진행중인 검색어 : 봉플라봉뱅


  0%|          | 0/79 [00:00<?, ?it/s]

리뷰수집중:   0%|          | 0/262 [00:00<?, ?it/s]


2
현재 :  스톤아일랜드탭하우스
현재 크롤링이 진행중인 검색어 : 스톤아일랜드탭하우스


  0%|          | 0/21 [00:00<?, ?it/s]

...

리뷰수집중:   0%|          | 0/238 [00:00<?, ?it/s]


5
현재 :  영화루
현재 크롤링이 진행중인 검색어 : 영화루


  0%|          | 0/9 [00:00<?, ?it/s]

리뷰수집중:   0%|          | 0/84 [00:00<?, ?it/s]


------------------------------------------------------------
종료


In [272]:
test_01.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 742 entries, 0 to 83
Data columns (total 11 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   리뷰        742 non-null    object
 1   리뷰이미지url  742 non-null    object
 2   작성자프로필    742 non-null    object
 3   My플레이스    742 non-null    object
 4   작성자이름     742 non-null    object
 5   리뷰작성수     742 non-null    object
 6   리뷰작성일     742 non-null    object
 7   식당방문수     742 non-null    object
 8   리뷰인증방식    742 non-null    object
 9   식당id      742 non-null    object
 10  사업장명      742 non-null    object
dtypes: object(11)
memory usage: 69.6+ KB


In [291]:
main = Crawling_review(data[60:])

In [292]:
main_frm = main.crawling_review_loop()

시작
------------------------------------------------------------
1
현재 :  동이트는집신제주점
현재 크롤링이 진행중인 검색어 : 동이트는집신제주점


  0%|          | 0/5 [00:00<?, ?it/s]

리뷰수집중:   0%|          | 0/56 [00:00<?, ?it/s]


2
현재 :  중원(반점)
현재 크롤링이 진행중인 검색어 : 중원(반점)


  0%|          | 0/7 [00:00<?, ?it/s]

...

리뷰수집중:   0%|          | 0/167 [00:00<?, ?it/s]


285
현재 :  검은쇠몰고오는
현재 크롤링이 진행중인 검색어 : 검은쇠몰고오는


  0%|          | 0/80 [00:00<?, ?it/s]

리뷰수집중:   0%|          | 0/263 [00:00<?, ?it/s]

NoSuchElementException: Message: no such element: Unable to locate element: {"method":"css selector","selector":".DqSJm > .place_thumb > img"}
  (Session info: chrome=113.0.5672.127)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00BC8893+48451]
	(No symbol) [0x00B5B8A1]
	(No symbol) [0x00A65058]
	(No symbol) [0x00A90467]
	(No symbol) [0x00A9069B]
	(No symbol) [0x00A89631]
	(No symbol) [0x00AAA304]
	(No symbol) [0x00A89586]
	(No symbol) [0x00AAA614]
	(No symbol) [0x00ABC482]
	(No symbol) [0x00AAA0B6]
	(No symbol) [0x00A87E08]
	(No symbol) [0x00A88F2D]
	GetHandleVerifier [0x00E28E3A+2540266]
	GetHandleVerifier [0x00E68959+2801161]
	GetHandleVerifier [0x00E6295C+2776588]
	GetHandleVerifier [0x00C52280+612144]
	(No symbol) [0x00B64F6C]
	(No symbol) [0x00B611D8]
	(No symbol) [0x00B612BB]
	(No symbol) [0x00B54857]
	BaseThreadInitThunk [0x768000C9+25]
	RtlGetAppContainerNamedObjectPath [0x77527B4E+286]
	RtlGetAppContainerNamedObjectPath [0x77527B1E+238]


In [297]:
# 344~
main_2 = Crawling_review(data[344:])

In [298]:
main_frm2 = main_2.crawling_review_loop()

시작
------------------------------------------------------------
1
현재 :  검은쇠몰고오는
현재 크롤링이 진행중인 검색어 : 검은쇠몰고오는


  0%|          | 0/81 [00:00<?, ?it/s]

리뷰수집중:   0%|          | 0/263 [00:00<?, ?it/s]


2
현재 :  피자쿠치나신제주점
현재 크롤링이 진행중인 검색어 : 피자쿠치나신제주점


  0%|          | 0/1 [00:00<?, ?it/s]

...

리뷰수집중:   0%|          | 0/168 [00:00<?, ?it/s]


213
현재 :  삼무국수
현재 크롤링이 진행중인 검색어 : 삼무국수


  0%|          | 0/259 [00:00<?, ?it/s]

리뷰수집중:   0%|          | 0/1559 [00:00<?, ?it/s]

ValueError: All arrays must be of the same length

In [301]:
#556~
main_3 = Crawling_review(data[556:])

In [302]:
main_frm3 = main_3.crawling_review_loop()

시작
------------------------------------------------------------
1
현재 :  삼무국수
현재 크롤링이 진행중인 검색어 : 삼무국수


  0%|          | 0/259 [00:00<?, ?it/s]

리뷰수집중:   0%|          | 0/1559 [00:00<?, ?it/s]


2
현재 :  피자마을앤파닭
현재 크롤링이 진행중인 검색어 : 피자마을앤파닭

3
현재 :  연동길
현재 크롤링이 진행중인 검색어 : 연동길


  0%|          | 0/32 [00:00<?, ?it/s]

...


412
현재 :  카페노연로117
현재 크롤링이 진행중인 검색어 : 카페노연로117


  0%|          | 0/8 [00:00<?, ?it/s]

리뷰수집중:   0%|          | 0/26 [00:00<?, ?it/s]


------------------------------------------------------------
종료


D:\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1179: UserWarning: Ignoring URL 'https://m.place.naver.com/my/5efdf8e467c6d4320f56f8a8/review?v=2' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
D:\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1179: UserWarning: Ignoring URL 'https://m.place.naver.com/my/6115ffe12f088f9a2540a13f/review?v=2' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
D:\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1179: UserWarning: Ignoring URL 'https://m.place.naver.com/my/5fc478cd3c1d179e95d8de37/review?v=2' since it exceeds Excel's limit of 65,530 URLS per worksheet.
  warn("Ignoring URL '%s' since it exceeds Excel's limit of "
D:\anaconda3\lib\site-packages\xlsxwriter\worksheet.py:1179: UserWarning: Ignoring URL 'https://m.place.naver.com/my/5d952cfc8f87a842bc7a5a11/review?v=2' since it exceed

In [260]:
def iframe_finder(iframe_id) :
    wait = WebDriverWait(driver, 10)
    driver.switch_to.default_content()
    # self.iframe
    iframe = wait.until(EC.presence_of_element_located((By.ID, iframe_id)))
    driver.switch_to.frame(iframe)
def is_action_successful(driver):
    try:
        wait = WebDriverWait(driver, 3)
        wait.until(EC.visibility_of_element_located((By.CLASS_NAME, "N_KDL")))
        return True
    except:
        return False
# 1. 식당이름 입력
# 2. iframe searchIframe 변경
# 3. 페이지에 식당이 있는지 확인
# 4. 내가 찾으려는 식당 주소와 검색해서 나온 식당 주소 비교 
# 5. 찾았다면 클릭

# Chrome headless 설정
options = Options()
# options.add_argument('--headless')

# 드라이버 생성
driver = webdriver.Chrome(service=Service(ChromeDriverManager().install()), options=options)

# name = '연화양꼬치'
# name = '맛깔참죽신제주점'
# name = '커피마마제주연동점'
# name = '삼성혈해물탕'
# name = '순창갈비'
# name = '스톤아일랜드탭하우스'
# name = '진낙지앤'
# name = '토크쇼코스모스점'
name = '봉플라봉뱅'
# 리뷰 페이지 열기
print(f'현재 크롤링이 진행중인 검색어 : {name}')

url = f"https://map.naver.com/v5/search/{name}"

driver.get(url)

wait = WebDriverWait(driver, 10)

# iframe_finder('searchIframe')
iframe_finder('entryIframe')

현재 크롤링이 진행중인 검색어 : 봉플라봉뱅


In [228]:
testdf

,리뷰,리뷰이미지url,작성자프로필,My플레이스,작성자이름,리뷰작성수,리뷰작성일,식당방문수,리뷰인증방식
0,None,None,None,None,None,None,None,None,None
1,None,None,None,None,None,None,None,None,None
2,None,None,None,None,None,None,None,None,None


In [152]:
import pandas as pd

a = [1,2,3,4,5]
b = ['a','b','c','c','d']

ab = pd.DataFrame({'a':a , 'b' : b})
ab['c'] = 1
ab

,a,b,c
0,1,a,1
1,2,b,1
2,3,c,1
3,4,c,1
4,5,d,1


In [156]:
def find_img_url(rest_img) :
    rest_img_urls = []
    for stylejs_parent in rest_img :
        stylejs = stylejs_parent.find_element(by = By.CSS_SELECTOR , value = 'a > .K0PDV._img.fKa0W')
        #print('stylejs = ' , stylejs)
        url_test_data = stylejs.get_attribute('style')
        #print('data = ' , url_test_data)
        url_find_num = url_test_data.find('url("')

        if 'jpg' in url_test_data :
            url_find_num_end = url_test_data.find('jpg')
        elif 'jpe' in url_test_data :
            url_find_num_end = url_test_data.find('peg')
        else :
            url_find_num_end = url_test_data.find('.")')

        rest_img_url = url_test_data[url_find_num + 5 : url_find_num_end+3]

        if rest_img_url != '' :
            rest_img_urls.append(rest_img_url)

    return rest_img_urls




iframe_finder('entryIframe')
review_text = []
review_image = []

review_all = driver.find_elements(by=By.CLASS_NAME , value = 'YeINN')
for review_once in review_all :
    
    rest_txt_case01 = review_once.find_elements(by=By.CLASS_NAME , value = 'ZZ4OK.IwhtZ > a > .zPfVt')
    rest_txt_case02 = review_once.find_elements(by=By.CLASS_NAME , value = 'ZZ4OK > a > .zPfVt')
    
    rest_img_case01 = review_once.find_elements(by=By.CLASS_NAME , 
                  value = 'Mmpzx > .ngGKH > .flicking-viewport > .flicking-camera > .ODalI')
    rest_img_case02 = review_once.find_elements(by=By.CLASS_NAME , 
                  value = 'Mmpzx.jQTnr > .ngGKH > .flicking-viewport > .flicking-camera > .ODalI')
    
    if len(rest_txt_case01) != 0 :
        review_text.append(rest_txt_case01[0].text)
    elif len(rest_txt_case02) != 0 :
        review_text.append(rest_txt_case02[0].text)
    else :
        review_text.append('')
        pass
        
    if len(rest_img_case01) != 0 :
        RIU = find_img_url(rest_img_case01)
        review_image.append(RIU)
    elif len(rest_img_case02) != 0 :
        RIU = find_img_url(rest_img_case02)
        review_image.append(RIU)
    else :
        review_image.append('') 

In [157]:
len(review_text)

10

In [154]:
len(review_image)

10

In [266]:
YeINN_tag[53].find_elements(by=By.CLASS_NAME , value='sb8UA > span')

[<selenium.webdriver.remote.webelement.WebElement (session="dcb7d9a010164a651f2c315766af1332", element="B13258A6BBD44DFFD6F51C6BC2378CF7_element_532")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dcb7d9a010164a651f2c315766af1332", element="B13258A6BBD44DFFD6F51C6BC2378CF7_element_533")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dcb7d9a010164a651f2c315766af1332", element="B13258A6BBD44DFFD6F51C6BC2378CF7_element_534")>,
 <selenium.webdriver.remote.webelement.WebElement (session="dcb7d9a010164a651f2c315766af1332", element="B13258A6BBD44DFFD6F51C6BC2378CF7_element_535")>]

In [261]:
# 닉네임 프로필 리뷰 수 사진 수 팔로워 수 가져오기
# 부모 태그
YeINN_tag = driver.find_elements(by=By.CLASS_NAME , value = 'YeINN')

In [179]:
# 작성자 프로필 사진
profile = YeINN_tag[0].find_element(by=By.CLASS_NAME , value ='DqSJm > .place_thumb > img')
profile.get_attribute('src')

'https://g-place.pstatic.net/assets/shared/images/icon_default_profile.png'

In [209]:
# 작성자 MY플레이스
my_place = YeINN_tag[0].find_element(by=By.CLASS_NAME , value ='Hazns')
my_place.get_attribute('href')

'https://m.place.naver.com/my/5da1f4ea8f87a842bc92ceed/review?v=2'

In [182]:
# 작성자 닉네임
nickname = YeINN_tag[0].find_element(by=By.CLASS_NAME , value ='Hazns > .sBWyy')
nickname.text

'쥴리지아니'

In [212]:
# 작성자 리뷰 사진 팔로워 수
reviewer_info = YeINN_tag[0].find_elements(by=By.CLASS_NAME , value ='Hazns > .Qde7Q > .P1zUJ')
reviewer_info_text_lst = []

if len(reviewer_info) != 0 :
    for reviewer_info_text in reviewer_info :
            #print(reviewer_info_text.text)
            reviewer_info_text_lst.append(reviewer_info_text.text)
else : 
    pass
reviewer_info_text_lst

['리뷰 419', '사진 725', '팔로워 46']

In [190]:
# 작성일자, 방문 횟수 가져오기

In [264]:
# 작성일자
review_date = YeINN_tag[53].find_elements(by=By.CLASS_NAME , value ='sb8UA > .P1zUJ > .place_blind')
review_date[2].text

'2021년 4월 28일 수요일'

In [207]:
# 방문 횟수
review_visit_count = YeINN_tag[0].find_elements(by=By.CLASS_NAME , value ='sb8UA > .P1zUJ')
review_visit_count[1].text

'1번째 방문'